**This notebook is an exercise in the [Time Series](https://www.kaggle.com/learn/time-series) course.  You can reference the tutorial at [this link](https://www.kaggle.com/ryanholbrook/trend).**

---


In [ ]:
# Importamos librerias
from learntools.core import binder
binder.bind(globals())
from learntools.time_series.ex2 import *

from pathlib import Path
from learntools.time_series.style import * 

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression

In [ ]:
# Obtenemos el Dataset [Contiene datos de ventas mensuales para varias industrias minoristas en los Estados Unidos]
data_dir = Path('../input/ts-course-data/')
comp_dir = Path('../input/store-sales-time-series-forecasting')

retail_sales = pd.read_csv(
    data_dir / "us-retail-sales.csv",
    parse_dates=['Month'],
    index_col='Month',
).to_period('D')
food_sales = retail_sales.loc[:, 'FoodAndBeverage']
auto_sales = retail_sales.loc[:, 'Automobiles']

dtype = {
    'store_nbr': 'category',
    'family': 'category',
    'sales': 'float32',
    'onpromotion': 'uint64',
}
store_sales = pd.read_csv(
    comp_dir / 'train.csv',
    dtype=dtype,
    parse_dates=['date'],
    infer_datetime_format=True,
)
store_sales = store_sales.set_index('date').to_period('D')
store_sales = store_sales.set_index(['store_nbr', 'family'], append=True)
average_sales = store_sales.groupby('date').mean()['sales']

-------------------------------------------------------------------------------

# 1) Vistazo al dataset

Vistazo al dataset el cual contiene un cojunto de datos de ventas de minoristas en los EE.UU en millones de dolares vs tiempo.

In [ ]:
ax = food_sales.plot(**plot_params)
ax.set(title="Ventas de alimentos y bebidas en EE. UU.", ylabel="Millones de dolares");

Se realiza una gráfica de promedio móvil para estimar la tendencia de esta serie.

In [ ]:
# Promedio con parámetros apropiados para la estimación de tendencias
trend = food_sales.rolling(
    window=12,
    center=True,
    min_periods=6,
).mean()

# Check your answer
q_1.check()

# Graficamos la función de promedio sobre la función de ventas
ax = food_sales.plot(**plot_params, alpha=0.5)
ax = trend.plot(ax=ax, linewidth=3)

-------------------------------------------------------------------------------

# 2) Identificamos la tendencia

¿Qué tendencia polinomial de orden podría ser apropiada para la serie *Ventas de alimentos y bebidas*?

La curva ascendente en la tendencia sugiere que un polinomio de orden 2 (cuadrático) podría ser apropiado.

In [ ]:
trend = average_sales.rolling(
    window=365,
    center=True,
    min_periods=183,
).mean()

ax = average_sales.plot(**plot_params, alpha=0.5)
ax = trend.plot(ax=ax, linewidth=3)

# 3) Creamos una función de tendencia

Utilizamos la libreria `statsmodels` para importar `DeterministicProcess` para crear un conjunto de características para un modelo de tendencia cúbico. 

También creamos características para un pronóstico de 90 días.

In [ ]:
from statsmodels.tsa.deterministic import DeterministicProcess

y = average_sales.copy()

dp = DeterministicProcess(index=y.index, order=3)
X = dp.in_sample()
X_fore = dp.out_of_sample(steps=90)

# Check your answer
q_3.check()

In [ ]:
# Graficamos el resultado del modelo de tendencia
model = LinearRegression()
model.fit(X, y)

y_pred = pd.Series(model.predict(X), index=X.index)
y_fore = pd.Series(model.predict(X_fore), index=X_fore.index)

ax = y.plot(**plot_params, alpha=0.5, title="Promedio de ventas", ylabel="Artículos vendidos")
ax = y_pred.plot(ax=ax, linewidth=3, label="Trend", color='C0')
ax = y_fore.plot(ax=ax, linewidth=3, label="Trend Forecast", color='C3')
ax.legend();

--------------------------------------------------------------------------------

Una manera de ajustar más el modelo es aumentar el orden del polinomio que se utiliza.

Para obtener hacer el modelo más complejo intentamos usando un polinomio de orden 11.

In [ ]:
from statsmodels.tsa.deterministic import DeterministicProcess

dp = DeterministicProcess(index=y.index, order=11)
X = dp.in_sample()

model = LinearRegression()
model.fit(X, y)

y_pred = pd.Series(model.predict(X), index=X.index)

ax = y.plot(**plot_params, alpha=0.5, title="Promedio de ventas", ylabel="Artículos vendidos")
ax = y_pred.plot(ax=ax, linewidth=3, label="Trend", color='C0')
ax.legend();

# 4) Comprender los riesgos de pronosticar con polinomios de alto orden

Sin embargo, los polinomios de alto orden generalmente no se adaptan bien a los pronósticos. ¿Puedes adivinar por qué?

Esto es debido a que se sobre-entrena el modelo, en consecuencia, el desempeño de nuestro modelo en el dataset de validación es bajo y por ende los pronosticos son muy poco confiables.

En el siguiente gráfico podemos observar el pronostico de 90 dias usando un polinomio de orden 11.

In [ ]:
X_fore = dp.out_of_sample(steps=90)
y_fore = pd.Series(model.predict(X_fore), index=X_fore.index)

ax = y.plot(**plot_params, alpha=0.5, title="Promedio de ventas", ylabel="Artículos vendidos")
ax = y_pred.plot(ax=ax, linewidth=3, label="Trend", color='C0')
ax = y_fore.plot(ax=ax, linewidth=3, label="Trend Forecast", color='C3')
ax.legend();

--------------------------------------------------------------------------------

# (Opcional) Ajuste de tendencia con splines

Los *Splines* son una buena alternativa a los polinomios cuando desea adaptarse a una tendencia. El algoritmo *Multivariate Adaptive Regression Splines* (MARS) de la biblioteca `pyearth` es potente y fácil de usar.

In [ ]:
from pyearth import Earth

# El target y las caracteristicas son las mismas que usamos anteriormente
y = average_sales.copy()
dp = DeterministicProcess(index=y.index, order=1)
X = dp.in_sample()

# Entrenamos MARS
model = Earth()
model.fit(X, y)

y_pred = pd.Series(model.predict(X), index=X.index)

ax = y.plot(**plot_params, title="Promedio de ventas", ylabel="Artículos vendidos")
ax = y_pred.plot(ax=ax, linewidth=3, label="Trend")

Por lo general, es muy difícil (si no imposible) pronosticar tendencias complicadas como esta. Sin embargo, con los datos históricos, se pueden usar splines para aislar otros patrones en una serie temporal eliminando la tendencia.

In [ ]:
y_detrended = y - y_pred   # Remueve la tendencia de store_sales

y_detrended.plot(**plot_params, title="Ventas promedio sin tendencia");